# Introduction to cypher

## What is Cypher?

Cypher is a query language designed for graphs.

- Nodes are represented by parentheses `()`.
- Colon to signify the label(s), for example `(:Person)`.
- Relationships between nodes are written with two dashes, for example `(:Person)--(:Movie)`.
- The direction of a relationship is indicated using a greater than or less than symbol `<` or `>` , for example `(:Person)-→(:Movie)`.
- The type of the relationship is written using the square brackets between the two dashes: `[` and `]`, for example `[:ACTED_IN]`
- Properties drawn in a speech bubble are specified in a JSON like syntax. Properties in Neo4j are key/value pairs, for example `{name: 'Tom Hanks'}`.

For example, a Cypher pattern in the graph could be:
```cypher
// example Cypher pattern
(m:Movie {title: 'Cloud Atlas'})<-[:ACTED_IN]-(p:Person)
```

## How Cypher works

Cypher works by matching patterns in the data. We retrieve data from the graph using the `MATCH` keyword. You can think of the `MATCH` clause as similar to the `FROM` clause in an SQL statement.

For example, if we want to find a Person in the graph, we would `MATCH` a pattern of a single node with a label of `:Person` - prefixed with a colon `:`.

```cypher
MATCH (p:Person)
RETURN p
```

Now, say we want to find the node which represents the Person who’s name is Tom Hanks. Our Person nodes all have a name property. We can use the braces `{..}` to specify the key/value pair of name and Tom Hanks as the filter. As Tom Hanks is a string, we will need to place it inside single or double quotes.

```cypher
MATCH (p:Person {name: 'Tom Hanks'})
RETURN p
```

In our Cypher statement, we can access properties using a dot notation. For example, to return the name property value using its property key p.name.

```cypher
MATCH (p:Person {name: 'Tom Hanks'})
RETURN  p.born
```

Another way that you can filter queries is by using the `WHERE` clause, rather than specifying the property value inline with braces.

```cypher
MATCH (p:Person)
WHERE p.name = 'Tom Hanks'
RETURN p.born
```

## Finding Relationships

We can extend the pattern in the MATCH clause to traverse through all relationships with a type of `ACTED_IN` to any node. Our domain model shows that the `ACTED_IN` relationship goes in an outgoing direction from the Person node so we can add the direction in our pattern. We often refer to this as a **traversal**.

```cypher
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->()
```

Our data model dictates that the node at the other end of that relationship will be Movie node, so we don’t necessarily need to specify the `:Movie` label in the node - instead we will use the variable m.

```cypher
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m)
RETURN m.title
```

If our graph had different labels, for example Television and Movie nodes this query would have returned all Television and Movie nodes that Tom Hanks acted in. That is, if we had multiple types of nodes at the end of the ACTED_IN relationships in our graph, we could make sure that we only return movies.

```cypher
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie)
RETURN m.title
```

## Filtering Queries

You have already learned how you can test equality for properties of a node and how you can use logical expressions to further filter what you want to retrieve.

### Filtering by node values

For example, this query retrieves the Person nodes and Movie nodes where the person acted in a movie that was released in 2008 or 2009:

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.released = 2008 OR m.released = 2009
RETURN p, m
```

### Filtering by node labels

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title='The Matrix'
RETURN p.name
```

An alternative to this query is the following where we test the node labels in the WHERE clause:

```cypher
MATCH (p)-[:ACTED_IN]->(m)
WHERE p:Person AND m:Movie AND m.title='The Matrix'
RETURN p.name
```

### Filtering using ranges

You can specify a range for filtering a query. Here we want to retrieve Person nodes of people who acted in movies released between 2000 and 2003:

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE 2000 <= m.released <= 2003
RETURN p.name, m.title, m.released
```

### Filtering by existence of a property

Recall that by default, there is no requirement that a node or relationship has a given property. Here is an example of a query where we only want to return Movie nodes where Jack Nicholson acted in the movie, and the movie has the tagline property.

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name='Jack Nicholson' AND m.tagline IS NOT NULL
RETURN m.title, m.tagline
```

### Filtering by partial strings

Cypher has a set of string-related keywords that you can use in your `WHERE` clauses to test string property values. You can specify `STARTS WITH`, `ENDS WITH`, and `CONTAINS`.

For example, to find all actors in the graph whose first name is Michael, you would write:

```cypher
MATCH (p:Person)-[:ACTED_IN]->()
WHERE p.name STARTS WITH 'Michael'
RETURN p.name
```

String tests are case-sensitive so you may need to use the `toLower()` or `toUpper()` functions to ensure the test yields the correct results. For example:

```cypher
MATCH (p:Person)-[:ACTED_IN]->()
WHERE toLower(p.name) STARTS WITH 'michael'
RETURN p.name
```

### Filtering by patterns in the graph

Suppose you wanted to find all people who wrote a movie but did not direct that same movie. Here is how you would perform the query:

```cypher
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT exists( (p)-[:DIRECTED]->(m) )
RETURN p.name, m.title
```

### Filtering using lists

If you have a set of values you want to test with, you can place them in a list or you can test with an existing list in the graph. A Cypher list is a comma-separated set of values within square brackets.

```cypher
MATCH (p:Person)
WHERE p.born IN [1965, 1970, 1975]
RETURN p.name, p.born
```

You can also compare a value to an existing list in the graph.

```cypher
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  'Neo' IN r.roles AND m.title='The Matrix'
RETURN p.name, r.roles
```

## What properties does a node or relationship have?

Discover the keys for the Person nodes in the graph by running this code:

```cypher
MATCH (p:Person)
RETURN p.name, keys(p)
```

## What properties exist in the graph?

More generally, you can run this code to return all the property keys defined in the graph.

```cypher
CALL db.propertyKeys()
```